In [4]:
import pandas as pd
import PyPDF2
import re
import json

from transformers import AutoModelForCausalLM, AutoModelForQuestionAnswering, AutoTokenizer, pipeline, BitsAndBytesConfig

import os
import torch
from langchain import PromptTemplate
from langchain.chains import SimpleSequentialChain, SequentialChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

import bitsandbytes

from PyPDF2 import PdfFileReader

In [ ]:
# Utility functions

# !!! Write a customized regex to extract the text without words like avv. n. etc.

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

def write_to_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)

def read_from_file(filename):
    with open(filename, 'r') as f:
        return f.read()
    
def split_text(text, pattern):
    parts = re.split(pattern, text, flags=re.MULTILINE)
    
    parts = [part for part in parts if part]
    
    if not re.match(pattern, parts[0]):
        parts = parts[1:]
    
    return parts

In [12]:
# Extract questions from the PDF with quiz

def extract_questions_answers(text):
    qa_pattern = re.compile(r'(\d+)\. (.+?)\n(A\) .+?)\n(B\) .+?)\n(C\) .+?)\n', re.DOTALL)
    matches = qa_pattern.findall(text)
    
    data = []
    for match in matches:
        question_number, question, answer_a, answer_b, answer_c = match
        
        # Remove newline characters and clean up answers
        question = question.replace('\n', ' ')
        answer_a = answer_a.replace('\n', ' ').replace('A) ', '')
        answer_b = answer_b.replace('\n', ' ').replace('B) ', '')
        answer_c = answer_c.replace('\n', ' ').replace('C) ', '')
        
        data.append([question, answer_a, answer_b, answer_c])
    
    return data

PC_PATH = "home/utente/Downloads/03_diritto_amministrativo.pdf"
REMOTE_PATH = "./03_diritto_amministrativo.pdf"
text = extract_text_from_pdf(REMOTE_PATH)
qa_data = extract_questions_answers(text)

df_pdf = pd.DataFrame(qa_data, columns=['Domanda', 'Risposta 1', 'Risposta 2', 'Risposta 3'])
df_pdf.to_csv('quiz_pdf.csv', index=False)

df_pdf.head()


,Domanda,Risposta 1,Risposta 2,Risposta 3
0,"Dal punto di vista degli effetti, le autorizza...",Consentono ad un soggetto di non adempiere ad ...,Permettono di esercitare facoltà preesistenti.,Accertano l'esistenza dei presupposti richiest...
1,Il Capo II della l. n. 241/1990 è riservato al...,"Accerta d'ufficio i fatti, disponendo il compi...",Non è mai competente alla valutazione della su...,É solo competente all'indizione delle conferen...
2,Con riferimento al riesame con esito demolitor...,Quanto a competenza a disporla spetta all'orga...,Non comparta in nessun caso l'obbligo di provv...,Quanto a competenza a disporla non spetta in n...
3,Ai sensi dell'art. 80 C.p.a. come avviene la p...,Deve essere presentata istanza di fissazione d...,Deve essere presentata istanza di fissazione d...,Deve essere presentato nuovamente il ricorso e...
4,Entro quale termine le parti devono proporre r...,Nel termine di sessanta giorni decorrente dall...,Nel termine di novanta giorni decorrente dalla...,Nel termine di centoventi giorni decorrente da...


In [15]:
# Extract questions from the JSON file

PC_PATH = "/home/utente/Downloads/domande.json"
REMOTE_PATH = "domande.json"
with open(REMOTE_PATH, 'r') as file:
    data = json.load(file)

questions = []
answers1 = []
answers2 = []
answers3 = []

# Iterate over each question object in the JSON data
for item in data:
    question = item['question']
    answers = item['answers']
    
    # Extract each answer and its correctness
    correct_answer = None
    incorrect_answers = []
    for answer in answers:
        if answer['right']:
            correct_answer = answer['answer']
        else:
            incorrect_answers.append(answer['answer'])
    
    answers1.append(correct_answer)
    answers2.append(incorrect_answers[0])
    answers3.append(incorrect_answers[1])
    
    questions.append(question)

df_json = pd.DataFrame({
    'Domanda': questions,
    'Risposta 1': answers1,
    'Risposta 2': answers2,
    'Risposta 3': answers3
})

df_json.to_csv('quiz_json.csv', index=False)

df_json.head()


,Domanda,Risposta 1,Risposta 2,Risposta 3
0,Ai sensi dell'art. 240 c.p. è sempre ordinata ...,dei beni che costituiscono il profitto del rea...,dei servizi e degli strumenti informatici o do...,delle cose che costituiscono il movente del reato
1,"Ai sensi dell'art. 266 c.p. è punito, se il fa...",fa a militari l'apologia di fatti contrari al ...,istiga gli incaricati di pubblico servizio a v...,istiga i funzionari pubblici a disobbedire ai ...
2,La pena della multa ex art. 24 c.p. consiste n...,non inferiore a euro 50 né superiore a euro 50...,non inferiore a euro 10 né superiore a euro 50...,non inferiore a euro 25 né superiore a euro 20...
3,Ai sensi dell'art. 7 c.p. è punito secondo la ...,Delitti commessi da pubblici ufficiali a servi...,Delitti contro la personalità del Presidente d...,Delitti di contraffazione della marchiatura de...
4,Tra le pene accessorie per i delitti ex art. 1...,la decadenza o la sospensione dall'esercizio d...,l'estinzione del rapporto di impiego subordina...,l'inabilitazione dagli uffici direttivi delle ...


In [16]:
# Merge the two dataframes and clean up the data

df_merged = pd.concat([df_pdf, df_json], ignore_index=True)
df_merged.to_csv('quiz_merged_orig.csv', index=False)

# Delete rows which don't contain a number (a law reference)
df_merged = df_merged[df_merged['Domanda'].apply(lambda x: bool(re.search(r'\d', x)))]
df_merged = df_merged.drop_duplicates(subset='Domanda')

# Add an index colum at the beginning of the dataframe
df_merged.insert(0, 'Index', range(1, 1 + len(df_merged)))

df_merged.to_csv('quiz_merged.csv', index=False)
df_merged.head()

,Index,Domanda,Risposta 1,Risposta 2,Risposta 3
1,1,Il Capo II della l. n. 241/1990 è riservato al...,"Accerta d'ufficio i fatti, disponendo il compi...",Non è mai competente alla valutazione della su...,É solo competente all'indizione delle conferen...
3,2,Ai sensi dell'art. 80 C.p.a. come avviene la p...,Deve essere presentata istanza di fissazione d...,Deve essere presentata istanza di fissazione d...,Deve essere presentato nuovamente il ricorso e...
4,3,Entro quale termine le parti devono proporre r...,Nel termine di sessanta giorni decorrente dall...,Nel termine di novanta giorni decorrente dalla...,Nel termine di centoventi giorni decorrente da...
7,4,A norma di quanto dispone l'art. 133 del C.p.a...,Alla giurisdizione esclusiva del giudice ammin...,Alla giurisdizione esclusiva del giudice ordin...,Alla giurisdizione esclusiva del TAR del Lazio.
8,5,Consacrando a livello costituzionale i princip...,"Moneta, tutela del risparmio e mercati finanzi...",Tutela e sicurezza del lavoro.,"Produzione, trasporto e distribuzione nazional..."


In [17]:
# Extract laws from a question (Mixtral-8x22B)
def ask_question(model, question, answers):
    #basic_prompt = "I am going to ask you a quiz question related to the legislative world, choose the correct answer between the possible options: "
    basic_prompt = "Sto per mostrarti una domanda a carattere legislativo, ricerca tutte le leggi all'interno della domanda e genera un JSON con i seguenti campi per ogni legge trovata {id_domanda,  numero della legge, testo della legge, link alla pagina web}"
    
    conversation = [
        {"role": "user", "content": basic_prompt + question},
        {"role": "assistant", "content": answers}
    ]
    
    tokenizer = AutoTokenizer.from_pretrained(model)
    tool_use_prompt = tokenizer.apply_chat_template (
        conversation,
        chat_template="tool_use",
        tools=tools,
        tokenize=False,
        add_generation_prompt=True,
    )
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x22B-Instruct-v0.1", device_map="cuda")

    inputs = tokenizer(tool_use_prompt, return_tensors="pt")

    outputs = model.generate(**inputs, max_new_tokens=20)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    
    return


In [16]:
# Extract laws from a question
# Function to quantize and load models
def load_quantized_model(model_name):
    bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                               )
        
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="cuda")
    return model, tokenizer

# Extract laws from a question
def extract_laws_from_question(model_data, quiz_list):
    answers = []
    model_pipeline = None
    
    for index, row in quiz_list.iterrows():
        
        print("<---- Question ---->\n")
        print(f"Index: {index}")
        print(f"question: {row['Domanda']}")#, answer_1: {row['Risposta 1']}, answer_2: {row['Risposta 2']}, answer_3: {row['Risposta 3']}")
        print("<---- Answers ---->\n")
        
        # Load the pipeline for question-answering
        if model_pipeline is None:
            model, tokenizer = load_quantized_model(model_data["model_name"])
            model_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

        # Ask the question to the model
        #question_prompt = "I am going to give you a question fro ma quiz about the legislative world. From that, extract all the laws and generate a JSON with the following fields for each law found: {law number, law text, link to the web page}."
        question_prompt = "Ti sottoporrò una domanda di un quiz sul mondo legislativo. Estraimi tutte i riferimenti a leggi che trovi e generare un JSON con i seguenti campi per ogni legge trovata: {numero della legge, testo della legge, link alla pagina web}. GENERA SOLAMENTE IL JSON, NESSUN TESTO AGGIUNTIVO."
        full_prompt = f"{question_prompt}\n\nDomanda: {row['Domanda']}"
        print(f"\nFull prompt: {full_prompt}")
        
        answer = model_pipeline(full_prompt, max_length=512, do_sample=True, top_p=0.95, num_return_sequences=1)
        print(f"answer -> {answer}")
        answers.append(answer[0]['generated_text'])
        
        """
        try:
            # Load the pipeline for question-answering
            if model_pipeline is None:
                model, tokenizer = load_quantized_model(model_data["model_name"])
                model_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

            # Ask the question to the model
            question_prompt = "I am going to give you a quiz question about the legislative world, from that extract all the laws and generate a JSON with the following fields for each law found {law number, law text, link to the web page}"
            answer = model_pipeline(question = question_prompt, context = row['Domanda'])
            print(f"model_data -> {model_data}")
            print(f"row -> {row}")
            print(f"answer -> {answer}")
            answers.append(answer["answer"])
        except Exception as e:
            answers.append(f"An error occurred: {str(e)}")
        print(f"{model_data["model_name"]} - {answers[-1]}")"""
    return answers

# List of the used models
models = {
    #"LegalBert": {'model_name': 'pile-of-law/legalbert-large-1.7M-2', 'context_window': 512}, #Modello molto rapido addestrato su testi legali -> Pessime performance per questa task + non supporta il text-generation
    #"Saul": {'model_name': 'Equall/Saul-7B-Instruct-v1', 'context_window': 1024}, #Modello addestrato su testi legali
    "Meta-Llama": {'model_name': 'meta-llama/Meta-Llama-3-8B', 'context_window': 2048},
    #"Falcon-7B": {'model_name': 'tiiuae/falcon-7b', 'context_window': 512},
    #"Mixtral-8x22B": {'model_name': 'mistralai/Mixtral-8x22B-Instruct-v0.1', 'context_window': 1024}, -> Non supporta il question answering
    #"Minerva-3B": {'model_name': 'sapienzanlp/Minerva-3B-base-v1.0', 'context_window': 512}, # Modello italiano della Sapienza
}

df = pd.read_csv('quiz_merged.csv')
output_models = []
output_answers = []

for model_data_key in models.keys():
    model_data = models[model_data_key]
    print(f"Model: {model_data['model_name']}")    
    output_answers += extract_laws_from_question(model_data, df)
    output_models += [models[model_data_key]["model_name"]] * len(df.index)

# Convert the dictionary to a DataFrame
df_extracted_laws = pd.DataFrame({
    'Model': output_models,
    'Answer': output_answers
})

# Rename the columns
df_extracted_laws.reset_index(inplace=True)
df_extracted_laws.columns = ['Model', 'Question Index', 'Extracted Laws']

# Split the 'Model' column into two separate columns
df_extracted_laws[['Model', 'Question Index']] = pd.DataFrame(df_extracted_laws['Model'].tolist(), index=df_extracted_laws.index)

print(df_extracted_laws)


Model: meta-llama/Meta-Llama-3-8B
<---- Question ---->

Index: 0
question: Il Capo II della l. n. 241/1990 è riservato alla regolamentazione della figura del  responsabile del procedimento, ovvero del  soggetto al quale è affidato il delicato ruolo di autorità di guida di ciascun procedimento  amministrativo. Esso: 
<---- Answers ---->



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:17<00:00,  4.27s/it]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Full prompt: Ti sottoporrò una domanda di un quiz sul mondo legislativo. Estraimi tutte i riferimenti a leggi che trovi e generare un JSON con i seguenti campi per ogni legge trovata: {numero della legge, testo della legge, link alla pagina web}. GENERA SOLAMENTE IL JSON, NESSUN TESTO AGGIUNTIVO.

Domanda: Il Capo II della l. n. 241/1990 è riservato alla regolamentazione della figura del  responsabile del procedimento, ovvero del  soggetto al quale è affidato il delicato ruolo di autorità di guida di ciascun procedimento  amministrativo. Esso: 


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


answer -> [{'generated_text': "Ti sottoporrò una domanda di un quiz sul mondo legislativo. Estraimi tutte i riferimenti a leggi che trovi e generare un JSON con i seguenti campi per ogni legge trovata: {numero della legge, testo della legge, link alla pagina web}. GENERA SOLAMENTE IL JSON, NESSUN TESTO AGGIUNTIVO.\n\nDomanda: Il Capo II della l. n. 241/1990 è riservato alla regolamentazione della figura del  responsabile del procedimento, ovvero del  soggetto al quale è affidato il delicato ruolo di autorità di guida di ciascun procedimento  amministrativo. Esso:  a) non può essere disposto se non per ragioni di urgenza, di  straordinaria gravità o di  pubblico interesse  rilevante, ovvero quando il procedimento è di competenza esclusiva di un  organo o di un  ufficio diverso da quello cui è affidato il  responsabile del procedimento, ovvero quando il  responsabile del procedimento è  sostituito; b) deve essere comunicato al  cittadino interessato almeno 15 giorni prima della sua emana

...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


answer -> [{'generated_text': "Ti sottoporrò una domanda di un quiz sul mondo legislativo. Estraimi tutte i riferimenti a leggi che trovi e generare un JSON con i seguenti campi per ogni legge trovata: {numero della legge, testo della legge, link alla pagina web}. GENERA SOLAMENTE IL JSON, NESSUN TESTO AGGIUNTIVO.\n\nDomanda: Il diritto di accesso ai documenti amministrativi (art. 22, l. n. 241/1990), consiste: 1) nel diritto di chiunque di conoscere l'attività amministrativa e l'attività di cui all'art. 3, c. 1, n. 2, lett. b) del l. n. 241/1990; 2) nel diritto di accesso agli atti e documenti, anche se conservati in copia o in esemplare, che abbiano formato oggetto di attività amministrativa e che non siano comunque coperti da segreto o riservatezza; 3) nel diritto di accesso agli atti e documenti che abbiano formato oggetto di attività amministrativa, anche se conservati in copia o in esemplare, e che non siano comunque coperti da segreto o riservatezza, salvo che siano stati emanat

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


answer -> [{'generated_text': "Ti sottoporrò una domanda di un quiz sul mondo legislativo. Estraimi tutte i riferimenti a leggi che trovi e generare un JSON con i seguenti campi per ogni legge trovata: {numero della legge, testo della legge, link alla pagina web}. GENERA SOLAMENTE IL JSON, NESSUN TESTO AGGIUNTIVO.\n\nDomanda: A norma del disposto di cui al co. 8, art. 23, D.Lgs. n. 50/2016, quale progetto deve  essere, altresì, corredato da apposito  piano di manutenzione dell'opera e delle sue parti in relazione al ciclo di vita? "}]
<---- Question ---->

Index: 14
question: Con il varo del T.U. n. 165/2001 (c.d. TUPI) è stato possibile distinguere nettamente i  poteri degli organi di governo da quelli  dei dirigenti. Agli organi di governo spetta in particolare: 
<---- Answers ---->


Full prompt: Ti sottoporrò una domanda di un quiz sul mondo legislativo. Estraimi tutte i riferimenti a leggi che trovi e generare un JSON con i seguenti campi per ogni legge trovata: {numero della legg

KeyboardInterrupt: 

In [9]:
bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                               )
        
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", quantization_config=bnb_config, device_map="cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [01:24<00:00, 21.06s/it]


In [ ]:
# Extract laws from a file
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Function to segment text into paragraphs
def segment_text(text):
    paragraphs = text.split('\n\n')  # Assuming paragraphs are separated by double newlines
    return paragraphs

# Function to chunk text into manageable sizes for the model
def chunk_text(paragraphs, tokenizer, max_length=512):
    chunks = []
    current_chunk = []
    current_length = 0

    for paragraph in paragraphs:
        paragraph_length = len(tokenizer.tokenize(paragraph))
        if current_length + paragraph_length > max_length:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
        current_chunk.append(paragraph)
        current_length += paragraph_length

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# Function to identify and extract laws using a pre-trained model
def extract_laws(chunks, model, tokenizer):
    laws = []
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors='pt', truncation=True)
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)
        if predictions.item() == 1:  # Assuming label 1 is for legal texts
            laws.append(chunk)
    return laws

# Function to clean and organize the extracted laws
def clean_and_organize_laws(laws):
    cleaned_laws = [law.strip() for law in laws if law.strip()]
    return cleaned_laws

# Main function to extract laws from a PDF
def extract_laws_from_pdf(pdf_path):
    # Step 1: Load the PDF
    text = extract_text_from_pdf(pdf_path)
    
    # Step 2: Segment the Text
    paragraphs = segment_text(text)
    
    # Step 3: Load the Model and Tokenizer
    """bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                               )
        
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
    model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", quantization_config=bnb_config, device_map="cuda")"""
    
    # Step 4: Chunk the Text
    chunks = chunk_text(paragraphs, tokenizer)
    
    # Step 5: Identify and Extract Laws
    raw_laws = extract_laws(chunks, model, tokenizer)
    
    # Step 6: Clean and Organize the Extracted Text
    cleaned_laws = clean_and_organize_laws(raw_laws)
    
    return cleaned_laws

# Example usage
PC_PATH = "home/utente/Downloads/Codice penale.pdf"
REMOTE_PATH = "./Codice penale.pdf"
pdf_path = REMOTE_PATH
laws = extract_laws_from_pdf(pdf_path)
for law in laws:
    print(law)


In [ ]:
# Extract laws from a file
# Function to extract text from a PDF
def extract_text_from_pdf(filepath):
    with open(filepath, 'rb') as file:
        reader = PdfFileReader(file)
        text = ''
        for page_num in range(reader.numPages):
            page = reader.getPage(page_num)
            text += page.extract_text()
    return text

# Function to write text to a file
def write_to_file(filename, text):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

# Function to split text into chunks
def split_text(text, max_chunk_size=7000, chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(separators=[
        ".\n",
    ],
    chunk_size=max_chunk_size, 
    chunk_overlap=chunk_overlap)
    
    return text_splitter.create_documents([text])

# Function to map (summarize each chunk)
def map_summarize_chunks(chunks, map_prompt, llm):
    map_chain = SimpleSequentialChain([map_prompt, llm])
    chunk_summaries = [map_chain.invoke({"text": chunk.page_content}).content for chunk in chunks]
    return chunk_summaries

# Function to reduce (summarize the combined summaries)
def reduce_summary(chunk_summaries, reduce_prompt, llm):
    combined_text = " ".join(chunk_summaries)
    reduce_chain = SimpleSequentialChain([reduce_prompt, llm])
    final_summary = reduce_chain.invoke({"text": combined_text}).content
    return final_summary

def process_pdf(filepath, map_prompt, reduce_prompt, llm):
    text = extract_text_from_pdf(filepath)
    chunks = split_text(text)
    for i, chunk in enumerate(chunks):
        write_to_file(f'chunk{i}.txt', chunk.page_content)
    chunk_summaries = map_summarize_chunks(chunks, map_prompt, llm)
    final_summary = reduce_summary(chunk_summaries, reduce_prompt, llm)
    return final_summary

# Default templates
map_prompt_template = \
"""Summarize the following text:

{text}

Summary:"""
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

reduce_prompt_template = \
"""Summarize the following combined summaries:

{text}

Final Summary:"""
reduce_prompt = PromptTemplate(template=reduce_prompt_template, input_variables=["text"])

# Define the model and the pipeline with 8-bit quantization
model_name = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, quantization_config=bnb_config, device_map="cuda")

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

class HuggingFaceTextGenerationChain(Chain):
    def __init__(self, generator):
        super().__init__()
        self.generator = generator

    def _call(self, inputs):
        text = inputs['text']
        generated_text = self.generator(text, max_length=100, num_return_sequences=1)[0]['generated_text']
        return {"generated_text": generated_text}

hugging_face_chain = HuggingFaceTextGenerationChain(generator)

# Use the chain for summarization
result = hugging_face_chain.run({"text": "Tell me a story about artificial intelligence."})
print(result['generated_text'])

# Example other component
class DummyChain(Chain):
    def _call(self, inputs):
        text = inputs['text']
        # Perform some dummy operations
        processed_text = text.upper()
        return {"text": processed_text}

dummy_chain = DummyChain()
hugging_face_chain = HuggingFaceTextGenerationChain(generator)

# Combine chains
complex_chain = SequentialChain(chains=[dummy_chain, hugging_face_chain])

# Use the complex chain with an input
result = complex_chain.run({"text": "Describe the future of technology."})
print(result['generated_text'])

reduce_prompt = PromptTemplate(template=reduce_prompt_template, input_variables=["text"])

# Path to PDF documents
path = "/home/utente/Desktop/Thesis/Documents/Downloaded"

for filename in os.listdir(path):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(path, filename)
        final_summary = process_pdf(pdf_path, map_prompt, reduce_prompt, hugging_face_chain)
        #write_to_file('summary.txt', final_summary)
        
        laws_json = complex_chain.invoke({"text": final_summary})
        print(f"Extracted JSON: {laws_json}")
